# Лабораторная работа 6

Задачи:

* Провести классификацию русских текстов на несколько категорий. Лучше большой текст
* Провести предобработку текстов: нормализацию, лемматизацию и тд
* Сравнить эмбеддинги
* Попробовать несколько методов классификации (+косинусы)

Будем использовать Войну и мир, Краткую историю всего

In [1]:
# будем делить Войду и мир, Краткую историю всего на partsCount одинаковых частей
partsCount = 30
myTestSize = 0.2

textDir = "/home/alex/Downloads/bmstuML/texts/"

warFile = textDir + "voina_i_mir.txt"

istoriaFile = textDir + "kratkaya.txt"


with open(warFile, encoding="utf8") as f:
    war_text = f.read()
    
with open(istoriaFile, encoding="utf8") as f:
    istoria_text = f.read()

In [2]:
war_text[:300]

'\n\nЛев Толстой\n\nВойна и мир\n\nТома первый и второй\n\n\n\nВ.\xa0Шкловский «Война и мир» Льва Толстого\n\n\nЗамысел\n\nВ 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге восходящего солнца были изображены пять портретов казненных декабристов; под портретами топор и подписано: «'

In [3]:
istoria_text[:300]

'\n«Краткая история почти всего на свете» Билла Брайсона — самая необычная энциклопедия из всех существующих! И это первая книга, которой была присуждена престижная европейская премия за вклад в развитие мировой науки имени Рене Декарта.\n\n\n\nСейчас, в начале 2003 года, я держу перед собой несколько стр'

### Токенизация

Выделим из текстов отдельные слова. Удалим ненужные "слова" (знаки препинания, лишние символы)


In [4]:
import nltk

nltk.download("punkt")
war_tokens = nltk.word_tokenize(war_text)

war_tokens[:15]

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Лев',
 'Толстой',
 'Война',
 'и',
 'мир',
 'Тома',
 'первый',
 'и',
 'второй',
 'В.',
 'Шкловский',
 '«',
 'Война',
 'и',
 'мир']

In [5]:
# второй текст
istoria_tokens = nltk.word_tokenize(istoria_text)

istoria_tokens[:15]

['«',
 'Краткая',
 'история',
 'почти',
 'всего',
 'на',
 'свете',
 '»',
 'Билла',
 'Брайсона',
 '—',
 'самая',
 'необычная',
 'энциклопедия',
 'из']

In [6]:
from snowballstemmer import RussianStemmer
stemmer = RussianStemmer()
stemmerResult = stemmer.stemWords(war_text.lower().split())

stemmerResult[:10]


['лев', 'толст', 'войн', 'и', 'мир', 'том', 'перв', 'и', 'втор', 'в.']

Как видим, слова получились "кривыми", простое обрезание окончания - не вариант. Воспользуемся лемматизатором.

In [7]:
import pymystem3

mystem = pymystem3.Mystem()

war_lemm = mystem.lemmatize(" ".join(war_tokens))

istoria_lemm = mystem.lemmatize(" ".join(istoria_tokens))



In [8]:
war_lemm[:20]

['лев',
 ' ',
 'толстой',
 ' ',
 'война',
 ' ',
 'и',
 ' ',
 'мир',
 ' ',
 'том',
 ' ',
 'первый',
 ' ',
 'и',
 ' ',
 'второй',
 ' ',
 'в',
 '. ']

In [9]:
istoria_lemm[:20]

['« ',
 'краткий',
 ' ',
 'история',
 ' ',
 'почти',
 ' ',
 'все',
 ' ',
 'на',
 ' ',
 'свет',
 ' » ',
 'билл',
 ' ',
 'брайсон',
 ' — ',
 'самый',
 ' ',
 'необычный']

Посмотрим на морфологический анализ слов.

In [10]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph.parse("свет")

[Parse(word='свет', tag=OpencorporaTag('NOUN,inan,masc,Sgtm sing,nomn'), normal_form='свет', score=0.51923, methods_stack=((DictionaryAnalyzer(), 'свет', 557, 0),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,inan,masc,Sgtm sing,accs'), normal_form='свет', score=0.423076, methods_stack=((DictionaryAnalyzer(), 'свет', 557, 4),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,anim,femn,Name plur,gent'), normal_form='света', score=0.01923, methods_stack=((DictionaryAnalyzer(), 'свет', 237, 8),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,anim,femn,Name plur,accs'), normal_form='света', score=0.01923, methods_stack=((DictionaryAnalyzer(), 'свет', 237, 10),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,anim,femn,Name sing,voct,Infr'), normal_form='света', score=0.01923, methods_stack=((DictionaryAnalyzer(), 'свет', 237, 13),))]

In [11]:
# Удалим знаки пустые слова

bad_chars = ["«", "»",".",",","—",":","1","2","3","4","5","6","7","8","9","0","?","-","[","]", "стр", "(", ")",
            ";", "X", "!", "издание", "…", "’","&"]

def containsStr(a,b):
    for i in b:
        if i in a:
            return True
    return False

def deleteWords(wordList):
    resultList = [item for item in wordList if len(item) > 1 ]
    resultList = [item for item in resultList if not containsStr(item, bad_chars) ]
    return resultList



war_lemm_clear = deleteWords(war_lemm)

istoria_lemm_clear = deleteWords(istoria_lemm)


In [12]:
war_lemm_clear[:20]

['лев',
 'толстой',
 'война',
 'мир',
 'том',
 'первый',
 'второй',
 'шкловский',
 'война',
 'мир',
 'лев',
 'толстой',
 'замысел',
 'год',
 'появляться',
 'объявление',
 'об',
 'полярный',
 'звезда',
 'на']

In [13]:
istoria_lemm_clear[:20]

['краткий',
 'история',
 'почти',
 'все',
 'на',
 'свет',
 'билл',
 'брайсон',
 'самый',
 'необычный',
 'энциклопедия',
 'из',
 'весь',
 'существующий',
 'это',
 'первый',
 'книга',
 'который',
 'быть',
 'присуждать']

In [14]:
war_text_clear = " ".join(war_lemm_clear)
istoria_text_clear = " ".join(istoria_lemm_clear)

In [15]:
war_text_clear[:300]

'лев толстой война мир том первый второй шкловский война мир лев толстой замысел год появляться объявление об полярный звезда на обложка книга круг восходить солнце быть изображать пять портрет казнить декабрист под портрет топор подписывать июль год том помечать день казнь декабрист над заглавие туч'

In [16]:
# сохраним текста

# with open(textDir+ '1.txt', 'w') as file:
#     file.write(war_text_clear)
    
# with open(textDir+ '2.txt', 'w') as file:
#     file.write(istoria_text_clear)  


In [17]:
# функция разделения на части
def chunkify(lst,n):
    return [lst[i::n] for i in range(n)]

In [18]:
import os
import pandas as pd
import numpy as np


war_parts = chunkify(war_lemm_clear, partsCount)
istoria_parts = chunkify(istoria_lemm_clear, partsCount)

istoria_parts[0][:10]

['краткий',
 'декарт',
 'изобретательность',
 'относиться',
 'мало',
 'повторяться',
 'из',
 'иан',
 'хичкок',
 'оффенбур']

In [19]:
# объединяем 
def joinWordsToTextList(wordsLists):
    result = []
    for i in wordsLists:
        oneText = " ".join(i)
        result.append(oneText)
    return result


war_texts = joinWordsToTextList(war_parts)
istoria_texts = joinWordsToTextList(istoria_parts)

In [20]:
war_texts[1][:300]

'толстой под изранить писать собр робкий ежели мой мыльный не пародировать государство год один на трещать характеристика герой что аракчеев путь три николай класс это воспоминание произведение среди близкий эпоха как объяснять обозначать семья иван часть вследствие толстой думать крайний только рома'

In [21]:
df = pd.DataFrame(columns=["text","class"])


for i in war_texts:
    df = df.append(pd.Series({"text":i, "class":0}), ignore_index=True)

for i in istoria_texts:
    df = df.append(pd.Series({"text":i, "class":1}), ignore_index=True)
    

print("war_texts size:"+str(len(war_texts[3])), "\tistoria_text size:"+str(len(istoria_texts[3])))


war_texts size:46816 	istoria_text size:34467


In [22]:
df

,text,class
0,лев декабрист солдат толстой полна человек рыл...,0
1,толстой под изранить писать собр робкий ежели ...,0
2,война портрет твердый из соч не мыльный россия...,0
3,мир топор как брюссель дальнейший мочь пузырь ...,0
4,том подписывать гранит герцен ссылка понимать ...,0
5,первый июль испытывать что на что лопнуть ясны...,0
6,второй год свой он этот лед для как попадать б...,0
7,шкловский том сила теперь приводиться трещать ...,0
8,война помечать так только непосредственно руши...,0
9,мир день же прочитывать текст под то рылеев ми...,0


In [23]:
# разбив на трейн и тест
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["text"], df["class"], test_size=myTestSize)

## Bag-of-Words-эмбеддинг
на всех данных обучние - чтоб все слова были учтены?

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

bof_vect = CountVectorizer()
bof_vect.fit(np.hstack([X_train, X_test]))
bof_train = bof_vect.transform(X_train)
bof_test = bof_vect.transform(X_test)

In [25]:
# пространство признаков

bof_train.toarray()

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [26]:
bof_train.toarray().shape

(48, 23015)

## TF-IDF-Эмбеддинг

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(np.hstack([X_train, X_test]))
tfidf_train = tfidf_vect.transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)


In [28]:
tfidf_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.00378749, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00731328, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00675776, 0.        ,
        0.        ]])

In [29]:
tfidf_train.toarray().shape

(48, 23015)

## Word2vec-эмбеддинг
Немного другой формат данных

In [30]:
from gensim.models import Word2Vec

X_train_w2v = X_train.apply(str.split)
X_test_w2v = X_test.apply(str.split)
w2v_vect = Word2Vec(np.hstack([X_train_w2v, X_test_w2v]))

In [31]:
X_train_w2v

27    [пять, спрашивать, герцен, только, человек, вс...
0     [лев, декабрист, солдат, толстой, полна, челов...
20    [обложка, манифест, том, не, писать, не, еще, ...
26    [изображать, кампания, познакомиться, кто, сил...
54    [вклад, не, тот, отмечать, героический, ричард...
7     [шкловский, том, сила, теперь, приводиться, тр...
10    [лев, казнь, подставлять, шестой, указание, но...
14    [появляться, туча, как, восторг, россия, что, ...
57    [наука, что, касательно, на, на, эллис, таймс,...
29    [казнить, севастопольский, март, толстой, этот...
18    [звезда, быть, соч, книга, все, один, пузырь, ...
13    [год, заглавие, палка, звезда, николаевский, д...
51    [европейский, отмечать, совсем, кто, любезност...
5     [первый, июль, испытывать, что, на, что, лопну...
2     [война, портрет, твердый, из, соч, не, мыльный...
53    [за, перига, далее, здесь, поистине, лондон, у...
11    [толстой, декабрист, свой, книга, том, этот, т...
15    [объявление, звезда, прежде, превосходный,

In [32]:
w2v_vect.most_similar(positive="год")

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('земля', 0.991139829158783),
 ('мы', 0.9608840346336365),
 ('если', 0.9538167119026184),
 ('до', 0.9258429408073425),
 ('вид', 0.9246758818626404),
 ('тысяча', 0.9080923795700073),
 ('время', 0.9060119390487671),
 ('по', 0.9048091173171997),
 ('более', 0.9048004150390625),
 ('самый', 0.9034135341644287)]

In [33]:
w2v_vect.most_similar(negative=["текст"])

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('ты', -0.5450881123542786),
 ('наташа', -0.5452762246131897),
 ('сказать', -0.5637020468711853),
 ('пьер', -0.5810374617576599),
 ('андрей', -0.5817877054214478),
 ('душенька', -0.586664617061615),
 ('она', -0.5878745317459106),
 ('князь', -0.5882515907287598),
 ('год', -0.6122918128967285),
 ('княжна', -0.6275584697723389)]

In [34]:
# как представлено слово Наташа
w2v_vect["наташа"]

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([ 0.5507305 ,  0.67350453, -0.24934688,  0.4454552 ,  1.685914  ,
        0.082981  , -0.24887723, -0.367418  ,  0.36407822,  0.6357176 ,
       -0.31058827, -0.56780154,  0.2013186 ,  0.73100334,  0.68598926,
       -0.6844782 ,  0.24531619,  0.12597807,  0.12873426,  0.14357343,
        0.5214272 , -0.9172658 ,  0.5530365 , -0.74684995,  0.21388508,
        1.0841601 , -0.23538832,  0.3047939 ,  0.14637445,  0.51675993,
        0.51090246,  1.3562737 ,  1.8032838 ,  0.6021039 , -0.13209277,
        0.68488777,  0.03495285,  0.8596143 ,  0.92250663, -0.7180531 ,
        0.60217   , -0.5469921 ,  0.16337858, -0.5986445 ,  0.4041892 ,
       -0.5659805 , -0.43969852,  0.48429614, -0.6750977 , -0.9770326 ,
       -0.11523149, -0.8135256 ,  0.8823747 ,  1.3896174 , -0.1812314 ,
       -0.00965889,  0.72365505,  0.1416582 ,  0.08070442,  0.5991289 ,
       -0.45289192,  0.15261953,  1.3674932 , -1.1744913 , -0.44874692,
        1.1256884 , -0.72741574,  0.05510331, -0.38129258,  0.47

In [35]:
# усредним векторы слов
def text2vec(text):
    vecs = []
    for word in text:
        try:
            vecs.append(w2v_vect[word])
        except KeyError:
            pass
        return np.sum(vecs, axis=0) / len(vecs)

    
    
w2v_train = X_train_w2v.apply(text2vec)
w2v_test = X_test_w2v.apply(text2vec)

w2v_train

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


27    [-0.14450927, 0.07438644, 0.25057772, 0.001098...
0     [-0.012421524, 0.007526146, 0.02644584, 0.0022...
20                                                  NaN
26    [-0.09343083, 0.05059665, 0.16415134, -0.00430...
54    [-0.0617709, 0.013402337, 0.10507288, -0.00649...
7                                                   NaN
10    [-0.012421524, 0.007526146, 0.02644584, 0.0022...
14    [-0.29240718, 0.050909456, 0.42542592, -0.0369...
57    [-0.37809783, -0.012650042, 0.49503136, -0.076...
29    [-0.02670082, 0.022555731, 0.056794405, -0.000...
18    [-0.2501593, 0.05255475, 0.37357274, -0.025573...
13    [-0.93176603, -0.42990717, 0.9347478, -0.32908...
51    [-0.06518073, 0.03434562, 0.11545077, 0.000774...
5     [-0.2732223, 0.04063116, 0.40220943, -0.034149...
2     [-0.1505694, 0.13543874, 0.30505425, 0.0174698...
53    [-0.027555766, 0.23452178, 0.22015241, 0.08266...
11    [-0.16647264, 0.12476157, 0.31941226, 0.002111...
15    [-0.010279304, 0.007055841, 0.02412204, 0.

In [36]:
# w2v_train = w2v_train.drop([w2v_train.index[14] , w2v_train.index[14]])
# w2v_train = w2v_train.drop([w2v_train.index[34] , w2v_train.index[34]])
# w2v_train = w2v_train.drop([w2v_train.index[35] , w2v_train.index[35]])







In [37]:
#w2v_train = np.dstack(w2v_train)[0]

In [38]:
w2v_train

27    [-0.14450927, 0.07438644, 0.25057772, 0.001098...
0     [-0.012421524, 0.007526146, 0.02644584, 0.0022...
20                                                  NaN
26    [-0.09343083, 0.05059665, 0.16415134, -0.00430...
54    [-0.0617709, 0.013402337, 0.10507288, -0.00649...
7                                                   NaN
10    [-0.012421524, 0.007526146, 0.02644584, 0.0022...
14    [-0.29240718, 0.050909456, 0.42542592, -0.0369...
57    [-0.37809783, -0.012650042, 0.49503136, -0.076...
29    [-0.02670082, 0.022555731, 0.056794405, -0.000...
18    [-0.2501593, 0.05255475, 0.37357274, -0.025573...
13    [-0.93176603, -0.42990717, 0.9347478, -0.32908...
51    [-0.06518073, 0.03434562, 0.11545077, 0.000774...
5     [-0.2732223, 0.04063116, 0.40220943, -0.034149...
2     [-0.1505694, 0.13543874, 0.30505425, 0.0174698...
53    [-0.027555766, 0.23452178, 0.22015241, 0.08266...
11    [-0.16647264, 0.12476157, 0.31941226, 0.002111...
15    [-0.010279304, 0.007055841, 0.02412204, 0.

## Классификация текстов
Для каждого эмбеддинга вычислим по два суммарных вектора - для текстов истории и мира(вона и мир)

In [39]:
mir_mean_bof = np.sum( bof_train[y_train == 0], axis=0 )
mir_mean_bof.shape

(1, 23015)

In [40]:
istoria_mean_bof = np.sum( bof_train[y_train == 1], axis=0 )


#### Для TF-IDF

In [41]:
mir_mean_tfidf = np.sum(tfidf_train[y_train == 0], axis=0 )

istoria_mean_tfidf = np.sum(tfidf_train[y_train == 1], axis=0 )


mir_mean_tfidf.shape

(1, 23015)

#### Для Word2Vec

In [42]:

# mir_mean_w2v = np.sum(w2v_train[:,y_train==0], axis=0)

# istoria_mean_w2v = np.sum(w2v_train[:,y_train==1], axis=0)

# mir_mean_w2v.shape


In [43]:
# mir_mean_w2v

Построим датафреймы с результатами классификаций тестовых текстов. Решение - по косинусному расстоянию.

In [44]:
from scipy.spatial.distance import cosine

bof_mir = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=mir_mean_bof)
bof_istoria = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=istoria_mean_bof)

bof_results = pd.DataFrame ([
    bof_mir,
    bof_istoria,
    np.maximum(bof_mir, bof_istoria) == bof_istoria,
    y_test
], index=["mir", "istoria", "predict", "class"]).T.astype(np.float)

bof_results

,mir,istoria,predict,class
0,0.018832,0.190968,1.0,0.0
1,0.013855,0.190964,1.0,0.0
2,0.015256,0.217468,1.0,0.0
3,0.218037,0.038851,0.0,1.0
4,0.015418,0.199935,1.0,0.0
5,0.018033,0.184858,1.0,0.0
6,0.216267,0.036906,0.0,1.0
7,0.232957,0.037328,0.0,1.0
8,0.226318,0.041814,0.0,1.0
9,0.016145,0.203064,1.0,0.0


In [45]:
from sklearn.metrics import accuracy_score

accuracy_score(bof_results["predict"], bof_results["class"])

0.0

In [46]:
tfidf_mir = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=mir_mean_tfidf)
tfidf_istoria = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=istoria_mean_tfidf)

tfidf_results = pd.DataFrame ([
    tfidf_mir,
    tfidf_istoria,
    np.maximum(tfidf_mir, tfidf_istoria) == tfidf_istoria,
    y_test
], index=["mir", "istoria", "predict", "class"]).T.astype (np.float)

tfidf_results

,mir,istoria,predict,class
0,0.049734,0.288472,1.0,0.0
1,0.045266,0.284744,1.0,0.0
2,0.046519,0.303924,1.0,0.0
3,0.347278,0.128319,0.0,1.0
4,0.046963,0.293790,1.0,0.0
5,0.050565,0.282672,1.0,0.0
6,0.345179,0.124254,0.0,1.0
7,0.358096,0.123501,0.0,1.0
8,0.352732,0.126258,0.0,1.0
9,0.048311,0.295477,1.0,0.0


In [47]:
accuracy_score(tfidf_results["predict"], tfidf_results["class"])

0.0

In [ ]:
# w2v не работает

#### Попробуем применить классическую модель поверх эмбеддингов

In [50]:
from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier().fit(tfidf_train.toarray(), y_train.tolist()).score(tfidf_test.toarray(), y_test.tolist())

1.0

In [51]:
RandomForestClassifier().fit(bof_train.toarray(), y_train.tolist()).score(bof_test.toarray(), y_test.tolist())

1.0